# 1 Jahr Frauenparlament: Sammeln der Daten zu Motionen

In [1]:
# import libraries
import pandas as pd
import numpy as np
import requests
import time
import math
import matplotlib.pyplot as plt

pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

#### API-Abfragen

In [86]:
# Motionen aus NR für Legislaturen 46 - 51 abfragen
motionen = pd.DataFrame()
legislaturen = [46, 47, 48, 49, 50, 51]
for legislatur in legislaturen:
    url = f"https://ws.parlament.ch/odata.svc/Business?$filter=BusinessTypes/ID eq 5 and Language eq 'DE' and SubmissionCouncilAbbreviation eq 'NR' and SubmissionLegislativePeriod eq {legislatur}&$format=json"
    response = requests.get(url) 
    df = pd.DataFrame(response.json()['d']['results'])
    motionen = motionen.append(df, ignore_index=True)
    while len(df) > 99:
        idmax = df.ID.max()
        url = f"https://ws.parlament.ch/odata.svc/Business?$filter=BusinessTypes/ID eq 5 and Language eq 'DE' and SubmissionCouncilAbbreviation eq 'NR' and SubmissionLegislativePeriod eq {legislatur} and ID gt {idmax}&$format=json"
        response = requests.get(url) 
        df = pd.DataFrame(response.json()['d']['results'])
        motionen = motionen.append(df, ignore_index=True)
    time.sleep(60)

In [87]:
# Datensatz auf relevante Informationen beschränken
motionen = motionen[['ID', 'Title', 'SubmittedText', 'ReasonText',
                     'SubmittedBy', 'BusinessStatusText', 'SubmissionCouncilName',
                     'SubmissionSession', 'SubmissionLegislativePeriod', 'TagNames']]

In [71]:
# Zwischenspeichern der Daten
motionen.to_pickle('internal/motionen.pkl')

In [93]:
## korrespondierende BusinessRole-Einträge anfragen (50 pro Durchlauf)
b_roles = pd.DataFrame()
for i in range(1,math.ceil(len(motionen)/50)+1):
    time.sleep(120)
    br_url = f"https://ws.parlament.ch/odata.svc/BusinessRole?$filter=(Businesses/ID eq {motionen.iloc[(i-1)*50,0]} "
    for i, motion in motionen.iloc[(i-1)*50+1:i*50,].iterrows():
        br_url += f"or Businesses/ID eq {motion['ID']} "
    br_url += ") and RoleName eq 'Urheber(-in)' and Businesses/Language eq 'DE'&$format=json"
    response = requests.get(br_url)
    b_roles = b_roles.append(pd.DataFrame(response.json()['d']['results']), ignore_index=True)

In [70]:
# Zwischenspeichern der Daten
b_roles.to_pickle('internal/businessroles.pkl')

In [15]:
## Informationen zu entsprechenden Nationalratsmitgliedern ermitteln
council_members = pd.DataFrame()
council_member_ids = pd.DataFrame(b_roles[~pd.isnull(b_roles.MemberCouncilNumber)]['MemberCouncilNumber'].unique(), columns=['MemberCouncilNumber'])
for i in [i for i in range(1, math.ceil(len(council_member_ids)/50)+1)]:
    mc_url = f"https://ws.parlament.ch/odata.svc/MemberCouncil?$filter=Language eq 'DE' and (ID eq {council_member_ids.iloc[(i-1)*50,0]} "
    for i, member in council_member_ids.iloc[(i-1)*50+1:i*50,].iterrows():
        mc_url += f"or ID eq {member['MemberCouncilNumber']} "    
    mc_url += ")&$format=json"
    response = requests.get(mc_url)
    council_members = council_members.append(pd.DataFrame(response.json()['d']['results']), ignore_index=True)
    time.sleep(120)

In [16]:
# Zwischenspeichern der Daten
council_members.to_pickle('internal/council_members.pkl')

#### Daten neu Laden

In [2]:
b_roles = pd.read_pickle('internal/businessroles.pkl')
council_members = pd.read_pickle('internal/council_members.pkl')
motionen = pd.read_pickle('internal/motionen.pkl')

#### Daten zusammenführen

In [19]:
pers_info = b_roles[['BusinessNumber', 'MemberCouncilNumber']].merge(council_members[['ID', 'FirstName',
                                                                                      'LastName', 'GenderAsString',
                                                                                      'CantonAbbreviation', 'CouncilAbbreviation',
                                                                                      'ParlGroupAbbreviation', 'PartyAbbreviation']],
                                                                     left_on='MemberCouncilNumber',
                                                                     right_on='ID',
                                                                     how='left')

In [20]:
full_data = motionen.merge(pers_info, left_on='ID', right_on='BusinessNumber', how='left')

#### Zusatzinformationen (Beitrittsdatum) für relevante NR-Mitglieder anfragen

In [21]:
council51 = pd.DataFrame(full_data[(full_data.SubmissionLegislativePeriod > 50) & ~(pd.isnull(full_data.MemberCouncilNumber))].MemberCouncilNumber.unique(), columns=['MemberCouncilNumber'])
for i in [1, 2, 3]:
    mc_history_url = f"https://ws.parlament.ch/odata.svc/MemberCouncilHistory?$filter=Language eq 'DE' and (PersonNumber eq {council51.iloc[i-1,0]}" 
    for i, member in council51.iloc[(i-1)*50+1:i*50,].iterrows():
        mc_history_url += f"or PersonNumber eq {member['MemberCouncilNumber']} "   
    mc_history_url += ") and CouncilAbbreviation eq 'NR'&$format=json"
    response = requests.get(mc_history_url)
    temp = pd.DataFrame(response.json()['d']['results'])
    for personnumber in temp['PersonNumber'].unique():
        if len(temp[temp['PersonNumber'] == personnumber]) > 0:
            date_joining = pd.to_datetime(temp[temp['PersonNumber'] == personnumber]['DateJoining'].str[6:-2].astype(int)/1000, unit='s').min()
        else:
            date_joining = np.nan
        council51.loc[council51[council51.MemberCouncilNumber == personnumber].index[0], 'DateJoining'] = date_joining

In [22]:
data = full_data[~pd.isnull(full_data.MemberCouncilNumber)].merge(council51, on='MemberCouncilNumber', how='left')

#### Neugewählte bestimmen

In [23]:
data['NewMember'] = False
data.loc[data.DateJoining > '2019-12', 'NewMember'] = True

#### Daten speichern

In [24]:
# Daten zur weiteren Verarbeitung abspeichern
data.to_pickle('internal/data.pkl')